In [11]:
from Bio import SeqIO
import csv
import os
import os.path
from os import listdir
from os.path import isfile, join
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
                           
#TSS = int(59)
promoter_length = int(100)
resultfolder = ("/home/breena/Documents/Project/blastresult_folder/")
my_data = [f for f in listdir(resultfolder) if isfile(join(resultfolder, f))]



## Extracting the promoter region:
Using the blast results to create variables for contigs, strains, gene names and promoter sequence start and end points - because this is a folder of files there is a certain way for reading in folders and then creating a list from the files instead said folder. 

The 168fna folder read in as handle is a folder of genomes of different strains of Listeria monocytogenes that are available through previous lab work; these files were iterated through to find the names, if the name matched the contig name (defined previously) then the script reads into the master gene array file that I had prepared with all putative SigB-dependent genes present in Listeria monocytogenes, and from this file, any line that contained the gene name of interest was then iterated through to pull out the distance from the transcription start site (TSS) which was previously recorded using Geneious. 

A folder of genbank formatted files of these same 168 strains was then used to iterate through in order to find the record id that matched previously defined contig names and then if a hit was found the promoter_start and promoter_end points were recorded for both positive and complementary strands; these positions were used then to iterate through the files and gather the promoter sequences that were 50 bp in length. 

In [12]:
import os
from Bio import SeqIO
import csv
import os.path
from os import listdir
from os.path import isfile, join
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

path = "/home/breena/Documents/Project/blastresult_folder/"
data = [f for f in listdir(path) if isfile(join(path, f))]
for file in data:
    print(file)
#path_1 = ("/home/breena/Documents/Project/blastresult_folder/")
#files = os.listdir(path_1)
#for f in files:
    #print(f)
    promoter_length = int(50) #change this so that the promoter seq is only 50 bp instead of 200
    #with open(path + file, "r") as resultfile:
            file_path = (path + file).replace(".fasta_output.csv", "")
            #print(path)
            gene_name = os.path.basename(file_path)
            print(gene_name)
            output = open("SigB_P" + gene_name + ".fasta", "w+")
            for line in resultfile:
                contig = line.split("\t")[4]
                #print(contig)
                contig_name = contig.split("|")[2]
                #print(contig_name)
                strain_name = contig_name.split("_")[0]
                #print(strain_name)
                start = int(line.split("\t")[2])
                #print(start)
                end = int(line.split("\t")[3])
                #print(end)
                if (int(end) - int(start)) >=0:
                    hit_strand = "+"
                else:
                    hit_strand = "-"
                with open("/home/breena/Documents/Project/168fna/" + strain_name + ".fna") as handle:
                    for record in SeqIO.parse(handle, "fasta"):
                    #print(record.id)
                    #print(repr(record.seq))
                    #print(len(record))
                        name = (record.id).split("|")[2]
                        #print(name)
                        if name == contig_name:
                            #print("contig found " + contig_name + hit_strand)
                            with open("/home/breena/Documents/Project/genes.csv", "r") as infile:
                                data_reader = csv.reader(infile)
                                for line in infile:
                                    #print(gene_name)
                                    if gene_name in line:
                                    #if line.startswith(gene_name):
                                        distance = int(line.split(",")[3])
                                        #print(distance)
                                        #break
                                        #using genbank files from here - should be fasta?
                                        gbkfolder = "/home/breena/Documents/Project/168gbk/"
                                        #my_data = [f for f in listdir(gbkfolder) if isfile(join(gbkfolder, f))]
                                        #the distance, gene_name should be taken from each row in the database file (columns A and E)
                                        #for file in my_data:
                                            #print(file)
                                        for rec in SeqIO.parse(gbkfolder + strain_name + ".gbk", "genbank"):
                                            #print(rec.id)
                                            if rec.id == contig_name:
                                                if hit_strand == "+":
                                                    promoter_start = int(start - distance - promoter_length)
                                                    promoter_end = int(start - distance)
                                                    #promoter = SeqFeature(FeatureLocation(280913,282107), type = "SigB-promotor", strand = 1)
                                                    promoter = SeqFeature(FeatureLocation(promoter_start, promoter_end),
                                                                          type="SigB-" + gene_name + "-promotor", strand = 1)
                                                    #print(promoter)
                                                    promoter_seq = (promoter.location.extract(rec).seq)
                                                    #print(promoter_seq)                                             
                                                else:
                                                    promoter_start = int(start + distance + promoter_length)
                                                    promoter_end = int(start + distance)
                                                    promoter = SeqFeature(FeatureLocation(promoter_end, promoter_start),
                                                                          type="SigB-" + gene_name + "-promotor", strand = -1)
                                                    #print(promoter)
                                                    promoter_seq = promoter.location.extract(rec).seq
                                                    #print(promoter_seq)
                                                    #promotor = SeqFeature(FeatureLocation(promoter_start, promoter_end), strand="+")
                                                        #promotors.append(promotor.location.extract(rec).seq
                                                #option 1 for writing output to file

                                                output.write(str('>' + strain_name.replace(".fna","") + '_SigB_P' + gene_name + '\n' + promoter_seq + '\n'))

            output.close() 

lmo0211
